# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  May 24 2022  3:42PM  238007        19  ACG-2102490166   
1  May 24 2022  3:41PM  238006        18        HH-34519   
2  May 24 2022  3:31PM  238005        22          615741   
3  May 24 2022  2:52PM  238003        10    MET209997-BO   
4  May 24 2022  2:52PM  238003        10    MET210197-BO   
5  May 24 2022  2:52PM  238003        10    MET210256-BO   
6  May 24 2022  2:52PM  238003        10       MET210493   
7  May 24 2022  2:52PM  238003        10       MET210494   
8  May 24 2022  2:52PM  238003        10       MET210497   
9  May 24 2022  2:52PM  238003        10       MET210498   

                Customer ShipmentStatus  
0  ACG North America LLC       Released  
1              Hush Hush       Released  
2      NBTY Global, Inc.       Released  
3       Methapharm, Inc.       Released  
4       Methapharm, Inc.       Released  
5       Methapharm, Inc.       Released  
6       Methapharm, Inc.       Released  
7       Methapharm, Inc.       Released  
8       Methapharm, Inc.       Released  
9       Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
31  238002       Released          1
32  238003       Released         21
33  238005       Released          1
34  238006       Released          1
35  238007       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238002                NaN        2.0       1.0
238003                NaN        NaN      21.0
238005                NaN        NaN       1.0
238006                NaN        NaN       1.0
238007                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237856                0.0        0.0       1.0
237864                0.0        0.0      11.0
237893                0.0        8.0       2.0
237924                0.0        0.0       1.0
237933               15.0       10.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237856                  0          0         1
237864                  0          0        11
237893                  0          8         2
237924                  0          0         1
237933                 15         10         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               237856          0          0         1
1               237864          0          0        11
2               237893          0          8         2
3               237924          0          0         1
4               237933         15         10         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               237856                            1
1               237864                           11
2               237893                   8        2
3               237924                            1
4               237933        15        10

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   May 24 2022  3:42PM  238007        19         ACG North America LLC
1   May 24 2022  3:41PM  238006        18                     Hush Hush
2   May 24 2022  3:31PM  238005        22             NBTY Global, Inc.
3   May 24 2022  2:52PM  238003        10              Methapharm, Inc.
24  May 24 2022  2:48PM  238002        19           AdvaGen Pharma, Ltd
27  May 24 2022  2:41PM  238001        10                   Emerginnova
28  May 24 2022  2:38PM  238000        10  Snap Medical Industries, LLC
30  May 24 2022  2:20PM  237997        10             ISDIN Corporation
40  May 24 2022  2:00PM  237993        21             NBTY Global, Inc.
41  May 24 2022  1:45PM  237989        20             Alumier Labs Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  May 24 2022  3:42PM  238007        19         ACG North America LLC   
1  May 24 2022  3:41PM  238006        18                     Hush Hush   
2  May 24 2022  3:31PM  238005        22             NBTY Global, Inc.   
3  May 24 2022  2:52PM  238003        10              Methapharm, Inc.   
4  May 24 2022  2:48PM  238002        19           AdvaGen Pharma, Ltd   
5  May 24 2022  2:41PM  238001        10                   Emerginnova   
6  May 24 2022  2:38PM  238000        10  Snap Medical Industries, LLC   
7  May 24 2022  2:20PM  237997        10             ISDIN Corporation   
8  May 24 2022  2:00PM  237993        21             NBTY Global, Inc.   
9  May 24 2022  1:45PM  237989        20             Alumier Labs Inc.   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3                           21  
4                   2        1  
5                            1  
6                   1        1  
7                   3        7  
8                            1  
9         8         3       37

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  May 24 2022  3:42PM  238007        19         ACG North America LLC   
1  May 24 2022  3:41PM  238006        18                     Hush Hush   
2  May 24 2022  3:31PM  238005        22             NBTY Global, Inc.   
3  May 24 2022  2:52PM  238003        10              Methapharm, Inc.   
4  May 24 2022  2:48PM  238002        19           AdvaGen Pharma, Ltd   
5  May 24 2022  2:41PM  238001        10                   Emerginnova   
6  May 24 2022  2:38PM  238000        10  Snap Medical Industries, LLC   
7  May 24 2022  2:20PM  237997        10             ISDIN Corporation   
8  May 24 2022  2:00PM  237993        21             NBTY Global, Inc.   
9  May 24 2022  1:45PM  237989        20             Alumier Labs Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3       21                      
4        1         2            
5        1                      
6        1         1            
7        7         3            
8        1                      
9       37         3         8

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  May 24 2022  3:42PM  238007        19  ACG North America LLC        1   
1  May 24 2022  3:41PM  238006        18              Hush Hush        1   
2  May 24 2022  3:31PM  238005        22      NBTY Global, Inc.        1   
3  May 24 2022  2:52PM  238003        10       Methapharm, Inc.       21   
4  May 24 2022  2:48PM  238002        19    AdvaGen Pharma, Ltd        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4         2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  May 24 2022  3:42PM  238007        19  ACG North America LLC       1.0   
1  May 24 2022  3:41PM  238006        18              Hush Hush       1.0   
2  May 24 2022  3:31PM  238005        22      NBTY Global, Inc.       1.0   
3  May 24 2022  2:52PM  238003        10       Methapharm, Inc.      21.0   
4  May 24 2022  2:48PM  238002        19    AdvaGen Pharma, Ltd       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        2.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  May 24 2022  3:42PM  238007        19  ACG North America LLC       1.0   
1  May 24 2022  3:41PM  238006        18              Hush Hush       1.0   
2  May 24 2022  3:31PM  238005        22      NBTY Global, Inc.       1.0   
3  May 24 2022  2:52PM  238003        10       Methapharm, Inc.      21.0   
4  May 24 2022  2:48PM  238002        19    AdvaGen Pharma, Ltd       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        2.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1427937      34.0       29.0        0.0
15          237933       0.0       10.0       15.0
16          475854       3.0        8.0        0.0
17          237924       1.0        0.0        0.0
18         1189745       5.0        0.0        0.0
19          951972      13.0       16.0        0.0
20          951810      68.0       10.0        8.0
21          475973       2.0        0.0        0.0
22          238005       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1427937      34.0       29.0        0.0
1        15   237933       0.0       10.0       15.0
2        16   475854       3.0        8.0        0.0
3        17   237924       1.0        0.0        0.0
4        18  1189745       5.0        0.0        0.0
5        19   951972      13.0       16.0        0.0
6        20   951810      68.0       10.0        8.0
7        21   475973       2.0        0.0        0.0
8        22   238005       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      34.0       29.0        0.0
1        15       0.0       10.0       15.0
2        16       3.0        8.0        0.0
3        17       1.0        0.0        0.0
4        18       5.0        0.0        0.0
5        19      13.0       16.0        0.0
6        20      68.0       10.0        8.0
7        21       2.0        0.0        0.0
8        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   34.0
1        15  Released    0.0
2        16  Released    3.0
3        17  Released    1.0
4        18  Released    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   17   18    19    20   21   22
Status                                                    
Completed   0.0  15.0  0.0  0.0  0.0   0.0   8.0  0.0  0.0
Executing  29.0  10.0  8.0  0.0  0.0  16.0  10.0  0.0  0.0
Released   34.0   0.0  3.0  1.0  5.0  13.0  68.0  2.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   17   18    19    20   21   22
0          Completed   0.0  15.0  0.0  0.0  0.0   0.0   8.0  0.0  0.0
1          Executing  29.0  10.0  8.0  0.0  0.0  16.0  10.0  0.0  0.0
2           Released  34.0   0.0  3.0  1.0  5.0  13.0  68.0  2.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   17   18    19    20   21   22
0  Completed   0.0  15.0  0.0  0.0  0.0   0.0   8.0  0.0  0.0
1  Executing  29.0  10.0  8.0  0.0  0.0  16.0  10.0  0.0  0.0
2   Released  34.0   0.0  3.0  1.0  5.0  13.0  68.0  2.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()